In [1]:
"""
NOTEBOOK TESTING CELLS
======================
Copy each section below into separate cells in m.ipynb.
"""

# ============================================================
# CELL 1: IMPORTS AND SETUP
# ============================================================
import sys
import os
import asyncio
from google.genai import types, client
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import ToolContext

# Add project root to path
sys.path.insert(0, os.path.abspath('.'))

from app.config import GEMINI_API_KEY, GEMINI_MODEL_NAME
from app.agents.initial_search_agent import initial_search_agent
from app.agents.parallel_extraction_agent import (
    leadership_agent,
    stakeholder_agent,
)
from app.models.citation import SourceDocument, SourceLibrary, FinalResponse, get_page_number

print("✅ Imports successful!")
print(f"📌 Model: {GEMINI_MODEL_NAME}")




2026-01-12 13:20:52.464 | INFO     | app.agents.parallel_extraction_agent:<module>:27 - 📋 Parallel agents configured with models: gemini-2.5-flash-lite / gemini-2.0-flash
2026-01-12 13:20:52.466 | DEBUG    | app.agents.parallel_extraction_agent:_create_extraction_agent:47 - 🔧 Creating extraction agent: LeadershipAgent -> output_key=leadership_data, model=gemini-2.5-flash-lite
2026-01-12 13:20:52.468 | DEBUG    | app.agents.parallel_extraction_agent:_create_extraction_agent:47 - 🔧 Creating extraction agent: MetricsAgent -> output_key=metrics_data, model=gemini-2.0-flash
2026-01-12 13:20:52.470 | DEBUG    | app.agents.parallel_extraction_agent:_create_extraction_agent:47 - 🔧 Creating extraction agent: StakeholderAgent -> output_key=stakeholder_data, model=gemini-2.5-flash-lite
2026-01-12 13:20:52.471 | DEBUG    | app.agents.parallel_extraction_agent:_create_extraction_agent:47 - 🔧 Creating extraction agent: ProductsAgent -> output_key=products_data, model=gemini-2.0-flash
2026-01-12 13:2

✅ Imports successful!
📌 Model: gemini-2.5-flash-lite


In [8]:
# Cell 1: Imports
import sys
import os
sys.path.insert(0, os.path.abspath('.'))

from google.genai import types
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from app.agents.sequential_extraction_agent import create_sequential_extraction_agent

# Cell 2: Run it
results = await run_extraction()

2026-01-12 12:34:19.524 | INFO     | app.agents.sequential_extraction_agent:<module>:23 - 📋 Sequential agents using model: gemini-2.5-flash-lite
2026-01-12 12:34:19.527 | DEBUG    | app.agents.parallel_extraction_agent:_create_extraction_agent:47 - 🔧 Creating extraction agent: LeadershipAgent -> output_key=leadership_data, model=gemini-2.5-flash-lite
2026-01-12 12:34:19.528 | DEBUG    | app.agents.parallel_extraction_agent:_create_extraction_agent:47 - 🔧 Creating extraction agent: MetricsAgent -> output_key=metrics_data, model=gemini-2.0-flash
2026-01-12 12:34:19.529 | DEBUG    | app.agents.parallel_extraction_agent:_create_extraction_agent:47 - 🔧 Creating extraction agent: StakeholderAgent -> output_key=stakeholder_data, model=gemini-2.5-flash-lite
2026-01-12 12:34:19.531 | DEBUG    | app.agents.parallel_extraction_agent:_create_extraction_agent:47 - 🔧 Creating extraction agent: ProductsAgent -> output_key=products_data, model=gemini-2.0-flash
2026-01-12 12:34:19.532 | DEBUG    | app.


⏳ Running 5 agents in parallel on Saksiam...


2026-01-12 12:34:22.483 | INFO     | app.tools:ask_annual_report:36 - 🔎 [Tool] Querying Store 'fileSearchStores/saksiam-store-x9i0i8eds9ex': Borrowers...
2026-01-12 12:34:26.448 | ERROR    | app.tools:ask_annual_report:209 - Tool Error: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-lite\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-lite\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash-lite\nPlease retry in 31.818444208s.',


✅ Done!
{"clean_text": "Tool Error: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-lite\\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-lite\\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash-lite\\nPlease retry in 31.818444208s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-a

  + Exception Group Traceback (most recent call last):
  |   File "c:\Users\HudaGoian\Documents\Cooperate Development\Company One Pager\Company_One_Pager\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3699, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "C:\Users\HudaGoian\AppData\Local\Temp\ipykernel_2744\960933609.py", line 12, in <module>
  |     results = await run_extraction()
  |               ^^^^^^^^^^^^^^^^^^^^^^
  |   File "C:\Users\HudaGoian\AppData\Local\Temp\ipykernel_2744\4256487655.py", line 50, in run_extraction
  |     async for event in events:
  |         if event.is_final_response() and event.content and event.content.parts:
  |             print(f"\n✅ Done!\n{event.content.parts[0].text[:1000]}")
  |   File "c:\Users\HudaGoian\Documents\Cooperate Development\Company One Pager\Company_One_Pager\.venv\Lib\site-packages\google\adk\runners.py", line 505, in run_async
  |     async for event in agen:
  |       yie

In [9]:
import os
from google import genai
from google.genai import types

# The client automatically picks up the API key from the environment variable (GEMINI_API_KEY)
client = genai.Client()

print("Available Gemini API Models:")
# Use the client.models.list() method to get a list of all models
for model in client.models.list():
    # You can access various attributes like name, description, input/output token limits, etc.
    print(f"Name: {model.name}")
    print(f"  Description: {model.description}")
    print(f"  Input Tokens: {model.input_token_limit}")
    print(f"  Output Tokens: {model.output_token_limit}")
    print("-" * 20)

Available Gemini API Models:
Name: models/embedding-gecko-001
  Description: Obtain a distributed representation of a text.
  Input Tokens: 1024
  Output Tokens: 1
--------------------
Name: models/gemini-2.5-flash
  Description: Stable version of Gemini 2.5 Flash, our mid-size multimodal model that supports up to 1 million tokens, released in June of 2025.
  Input Tokens: 1048576
  Output Tokens: 65536
--------------------
Name: models/gemini-2.5-pro
  Description: Stable release (June 17th, 2025) of Gemini 2.5 Pro
  Input Tokens: 1048576
  Output Tokens: 65536
--------------------
Name: models/gemini-2.0-flash-exp
  Description: Gemini 2.0 Flash Experimental
  Input Tokens: 1048576
  Output Tokens: 8192
--------------------
Name: models/gemini-2.0-flash
  Description: Gemini 2.0 Flash
  Input Tokens: 1048576
  Output Tokens: 8192
--------------------
Name: models/gemini-2.0-flash-001
  Description: Stable version of Gemini 2.0 Flash, our fast and versatile multimodal model for scalin

In [10]:
GEMINI_MODEL_NAME= "gemma-3-27b-it"

In [6]:
# ============================================================
# CELL 2: CONFIGURATION
# ============================================================
# Set your test parameters here
APP_NAME = "notebook_test"
USER_ID = "test_user"  
SESSION_ID = "notebook_session"

# OPTION A: Start fresh - upload a new PDF
COMPANY_NAME = "Saksiam"
PDF_PATH = "C:/Users/HudaGoian/Documents/Cooperate Development/Company One Pager/Saksiam AR 24.pdf"

# OPTION B: Use existing vector store (skip upload)
# Set this to your existing vector store name to skip the upload step
EXISTING_VECTOR_STORE = "fileSearchStores/saksiam-store-kwxr6sqe0z7j"

print(f"📁 Company: {COMPANY_NAME}")
print(f"📄 PDF: {PDF_PATH}")
print(f"🗄️ Existing Vector Store: {EXISTING_VECTOR_STORE or 'None (will create new)'}")



📁 Company: Saksiam
📄 PDF: C:/Users/HudaGoian/Documents/Cooperate Development/Company One Pager/Saksiam AR 24.pdf
🗄️ Existing Vector Store: fileSearchStores/saksiam-store-kwxr6sqe0z7j


In [21]:
def ask_annual_report(store_name: str, question: str):
    """
    Queries the currently loaded Annual Report PDF from the vector store.
    
    Returns a FinalResponse structure as a dictionary with:
        - clean_text: Text WITHOUT citation tags (for reading)
        - cited_text: Text WITH [[Src:xxx]] citation tags
        - citations: List of citation metadata with positions and source IDs
        - sources: Dictionary of SourceDocument objects for tooltips
    """
    if not store_name:
        return FinalResponse(
            clean_text="Error: No Annual Report vector store has been created or is available.",
            cited_text="Error: No Annual Report vector store has been created or is available.",
            citations=[],
            sources={}
        ).model_dump()

    time.sleep(2)  # To avoid rate limiting issues

    try: 
        gemini_client = client.Client(
            api_key=GEMINI_API_KEY,
        )
        response = gemini_client.models.generate_content(
            model= "gemini-2.5-flash-lite",
            contents=question,
            config=types.GenerateContentConfig(
                tools=[types.Tool(
                    file_search=types.FileSearch(
                        file_search_store_names=[store_name]
                    )
                )],
            )
        )
        return response
    except Exception as e:
        return FinalResponse(
            clean_text=f"Error querying Annual Report: {str(e)}",
            cited_text=f"Error querying Annual Report: {str(e)}",
            citations=[],
            sources={}
        ).model_dump()

In [22]:
gemini_client = client.Client(api_key=GEMINI_API_KEY)
stores = list(gemini_client.file_search_stores.list())
print(f"Found {len(stores)} stores.")

for store in stores:
    print(f"\n   Store: {store.name} ({store.display_name})")
    

Found 3 stores.

   Store: fileSearchStores/saksiam-store-t4oph360h8i0 (Saksiam Store)

   Store: fileSearchStores/saksiam-store-vnqa0lcxdl6s (Saksiam Store)

   Store: fileSearchStores/saksiam-store-kwxr6sqe0z7j (Saksiam Store)


In [23]:
import time
result = ask_annual_report(
    store_name="fileSearchStores/saksiam-store-vnqa0lcxdl6s",
    question="Who are the key shareholders and their ownership percentages in 2024?"
)
result

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            text="""As of December 31, 2024, the key shareholders and their ownership percentages are as follows:

*   **Mrs. Jintana Boonsalee:** 24.134%
*   **Mr. Phoonsak Boonsalee:** 23.903%
*   **Miss Sansanee Boonsalee:** 10.295%
*   **Mr. Siwaphong Boonsalee:** 9.661%

The Boonsalee Family collectively holds 67.993% of the shares.

Other major shareholders include:
*   **Bualuang Ventures Company Limited:** 7.156%
*   **K Mid Small Cap Equity RMF:** 1.518%
*   **Mr. Nopachai Nopasuwanwong:** 1.262%
*   **Thai NVDR Company Limited:** 1.178%
*   **Mr. Somkiat Kraikringsri:** 0.589%
*   **Mr. Aiyawatt Srivaddhanaprabha:** 0.573%

The total percentage held by minority shareholders is 12.276%.

Additionally, the following individuals are directors with specified shareholdings as of December 31, 2024:
*   **Mr. Suphot Singhasaneh:** 0.0

In [9]:
"""
NOTEBOOK TESTING CELLS
======================
Copy each section below into separate cells in m.ipynb.
"""

# ============================================================
# CELL 1: IMPORTS AND SETUP
# ============================================================
import sys
import os
import asyncio
from google.genai import types, client
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import ToolContext

# Add project root to path
sys.path.insert(0, os.path.abspath('.'))

from app.config import GEMINI_API_KEY, GEMINI_MODEL_NAME
from app.agents.initial_search_agent import initial_search_agent
from app.agents.parallel_extraction_agent import (
    leadership_agent,
    key_persons_agent,
    metrics_agent,
    stakeholder_agent,
    business_description_agent
)
from app.models.citation import SourceDocument, SourceLibrary, FinalResponse, get_page_number

print("✅ Imports successful!")
print(f"📌 Model: {GEMINI_MODEL_NAME}")


# ============================================================
# CELL 2: CONFIGURATION
# ============================================================
# Set your test parameters here
APP_NAME = "notebook_test"
USER_ID = "test_user"  
SESSION_ID = "notebook_session"

# OPTION A: Start fresh - upload a new PDF
COMPANY_NAME = "Saksiam"
PDF_PATH = "C:/Users/HudaGoian/Documents/Cooperate Development/Company One Pager/Company_One_Pager/Saksiam_annual_report.pdf"

# OPTION B: Use existing vector store (skip upload)
# Set this to your existing vector store name to skip the upload step
EXISTING_VECTOR_STORE = None  # e.g., "fileSearchStores/saksiam-store-903a6zfw3u20"

print(f"📁 Company: {COMPANY_NAME}")
print(f"📄 PDF: {PDF_PATH}")
print(f"🗄️ Existing Vector Store: {EXISTING_VECTOR_STORE or 'None (will create new)'}")


# ============================================================
# CELL 3: DIRECT TOOL TEST (ask_annual_report)
# ============================================================
"""
This cell lets you test the ask_annual_report tool DIRECTLY without running agents.
Useful for debugging tool behavior with an existing vector store.
"""
import time
from typing import Dict, Any

def test_ask_annual_report_direct(store_name: str, question: str) -> Dict[str, Any]:
    """
    Direct test of the ask_annual_report logic.
    Copy of the tool function without ToolContext dependency.
    """
    if not store_name:
        print("❌ No store_name provided!")
        return {"error": "No store_name"}
    
    print(f"🔎 Querying Store '{store_name}'")
    print(f"❓ Question: {question}")
    print("-" * 60)
    
    time.sleep(2)  # Rate limiting
    
    gemini_client = client.Client(api_key=GEMINI_API_KEY)
    
    response = gemini_client.models.generate_content(
        model="gemini-2.5-flash",
        contents=question,
        config=types.GenerateContentConfig(
            tools=[types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=[store_name]
                )
            )],
        )
    )
    
    print(f"\n📝 RESPONSE TEXT:\n{'-'*40}")
    print(response.text[:2000] if len(response.text) > 2000 else response.text)
    
    # Process grounding metadata
    source_library = SourceLibrary()
    source_counter = 100
    chunk_to_source_map: Dict[int, str] = {}
    
    if response.candidates:
        candidate = response.candidates[0]
        grounding = getattr(candidate, 'grounding_metadata', None)
        
        if grounding:
            grounding_chunks = getattr(grounding, 'grounding_chunks', None)
            
            if grounding_chunks:
                print(f"\n📋 GROUNDING CHUNKS ({len(grounding_chunks)} found):")
                print("=" * 60)
                
                for i, chunk in enumerate(grounding_chunks):
                    source_id = f"src_{source_counter}"
                    source_counter += 1
                    
                    retrieved_context = getattr(chunk, 'retrieved_context', None)
                    if retrieved_context:
                        title = getattr(retrieved_context, 'title', 'Unknown')
                        text = getattr(retrieved_context, 'text', '')
                        page_number = get_page_number(text)
                        
                        source_doc = SourceDocument(
                            source_id=source_id,
                            title=title,
                            page_number=page_number,
                            raw_text=text[:500] if text else ""
                        )
                        source_library.add_source(source_doc)
                        chunk_to_source_map[i] = source_id
                        
                        print(f"\n📄 CHUNK {i} ({source_id}):")
                        print(f"   📑 Title: {title}")
                        print(f"   📖 Page: {page_number}")
                        print(f"   📝 Text: {text[:200]}...")
            
            # Check grounding supports
            grounding_supports = getattr(grounding, 'grounding_supports', None)
            if grounding_supports:
                print(f"\n📍 GROUNDING SUPPORTS ({len(grounding_supports)} found):")
                print("=" * 60)
                
                for j, support in enumerate(grounding_supports):
                    segment = getattr(support, 'segment', None)
                    chunk_indices = getattr(support, 'grounding_chunk_indices', [])
                    
                    if segment:
                        seg_text = getattr(segment, 'text', '')
                        start_idx = getattr(segment, 'start_index', 0)
                        end_idx = getattr(segment, 'end_index', 0)
                        
                        source_ids = [chunk_to_source_map.get(ci, f"chunk_{ci}") for ci in chunk_indices]
                        
                        print(f"\n📍 SUPPORT {j}:")
                        print(f"   🔢 Indices: {start_idx}-{end_idx}")
                        print(f"   🔗 Sources: {source_ids}")
                        print(f"   📝 Text: '{seg_text[:100]}...'")
    
    return {
        "response_text": response.text,
        "sources": source_library.to_dict(),
        "chunk_map": chunk_to_source_map
    }

# TEST IT! Replace with your vector store name
# result = test_ask_annual_report_direct(
#     store_name="fileSearchStores/saksiam-store-903a6zfw3u20",
#     question="Who are the key shareholders and their ownership percentages?"
# )


# ============================================================
# CELL 4: RUN INITIAL SEARCH AGENT (Create Vector Store)
# ============================================================
async def run_initial_search():
    """Run initial_search_agent to upload PDF and create vector store."""
    session_service = InMemorySessionService()
    
    initial_state = {
        "company_name": COMPANY_NAME,
        "annual_report_filename": PDF_PATH
    }
    
    session = await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
        state=initial_state
    )
    
    print(f"📝 Session created with state: {initial_state}")
    
    runner = Runner(
        agent=initial_search_agent,
        app_name=APP_NAME,
        session_service=session_service
    )
    
    query = types.Content(role='user', parts=[types.Part(text="Find the annual report.")])
    
    events = runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=query)
    
    async for event in events:
        if event.is_final_response():
            print("\n✅ InitialSearchAgent Result:")
            if event.content and event.content.parts:
                print(event.content.parts[0].text)
    
    # Get updated session
    session = await session_service.get_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    vector_store_name = session.state.get("vector_store_name")
    
    print(f"\n✅ Vector Store Ready: {vector_store_name}")
    
    return session_service, vector_store_name

# Uncomment to run:
# session_service, vector_store_name = await run_initial_search()


# ============================================================
# CELL 5: RUN EXTRACTION AGENT WITH EXISTING STORE
# ============================================================
async def run_extraction_agent(agent, agent_name: str, question: str, vector_store_name: str, session_service=None):
    """
    Run any extraction agent with an existing vector store.
    
    Args:
        agent: The agent to run (leadership_agent, stakeholder_agent, etc.)
        agent_name: Name for logging
        question: Query to send to the agent
        vector_store_name: Existing vector store name
        session_service: Optional existing session service
    """
    if session_service is None:
        session_service = InMemorySessionService()
        
        # Create session with vector store already set
        session = await session_service.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=SESSION_ID,
            state={
                "company_name": COMPANY_NAME,
                "vector_store_name": vector_store_name  # Pre-set the vector store!
            }
        )
        print(f"✅ Created session with existing vector store: {vector_store_name}")
    
    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service
    )
    
    print(f"\n{'='*60}")
    print(f"🔄 Running {agent_name}")
    print(f"{'='*60}")
    
    query = types.Content(role='user', parts=[types.Part(text=question)])
    
    events = runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=query)
    
    async for event in events:
        if event.is_final_response():
            print(f"\n✅ {agent_name} Result:")
            if event.content and event.content.parts:
                result = event.content.parts[0].text
                print(result[:3000] if len(result) > 3000 else result)
    
    # Return updated session
    session = await session_service.get_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    return session_service, session.state

# Example: Run stakeholder_agent with existing vector store
# session_service, state = await run_extraction_agent(
#     agent=stakeholder_agent(),
#     agent_name="StakeholderAgent",
#     question="Extract all shareholders and their ownership percentages.",
#     vector_store_name="fileSearchStores/saksiam-store-903a6zfw3u20"
# )


# ============================================================
# CELL 6: QUICK TEST - All Agents Sequentially
# ============================================================
async def run_all_agents(vector_store_name: str):
    """Run all extraction agents in sequence with the same vector store."""
    
    agents_to_test = [
        (leadership_agent(), "LeadershipAgent", "Extract the management team information."),
        (stakeholder_agent(), "StakeholderAgent", "Extract all shareholders and their ownership percentages."),
        (metrics_agent(), "MetricsAgent", "Extract key financial metrics."),
        (business_description_agent(), "BusinessDescriptionAgent", "Describe the company's business."),
    ]
    
    results = {}
    session_service = InMemorySessionService()
    
    # Create initial session
    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
        state={
            "company_name": COMPANY_NAME,
            "vector_store_name": vector_store_name
        }
    )
    
    for agent, name, question in agents_to_test:
        try:
            _, state = await run_extraction_agent(
                agent=agent,
                agent_name=name,
                question=question,
                vector_store_name=vector_store_name,
                session_service=session_service
            )
            results[name] = "✅ Success"
        except Exception as e:
            results[name] = f"❌ Error: {e}"
            print(f"❌ {name} failed: {e}")
    
    print("\n" + "="*60)
    print("📊 SUMMARY")
    print("="*60)
    for name, status in results.items():
        print(f"  {name}: {status}")
    
    return results

# Uncomment to run all agents:
# results = await run_all_agents("fileSearchStores/saksiam-store-903a6zfw3u20")


# ============================================================
# CELL 7: RUN PARALLEL EXTRACTION AGENT ON EXISTING STORE
# ============================================================
"""
This cell runs the parallel_extraction_agent on an existing vector store.
All 5 sub-agents (Leadership, Metrics, Stakeholder, Products, Overview) 
run in parallel against the Saksiam annual report.
"""
from app.agents.parallel_extraction_agent import create_parallel_extraction_agent

# Configuration for existing store
SAKSIAM_STORE_NAME = "fileSearchStores/saksiam-store-x9i0i8eds9ex"
SAKSIAM_COMPANY_NAME = "Saksiam"

async def run_parallel_extraction_on_existing_store(
    store_name: str = SAKSIAM_STORE_NAME,
    company_name: str = SAKSIAM_COMPANY_NAME
):
    """
    Run the parallel extraction agent on an existing vector store.
    
    Args:
        store_name: The existing vector store name
        company_name: The company name for context
        
    Returns:
        Dictionary with all extracted data from session state
    """
    print(f"\n{'='*60}")
    print(f"🚀 PARALLEL EXTRACTION AGENT TEST")
    print(f"{'='*60}")
    print(f"📁 Company: {company_name}")
    print(f"🗄️ Vector Store: {store_name}")
    print(f"{'='*60}\n")
    
    session_service = InMemorySessionService()
    
    # Create session with pre-existing vector store
    initial_state = {
        "company_name": company_name,
        "vector_store_name": store_name
    }
    
    session = await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=f"{SESSION_ID}_parallel",
        state=initial_state
    )
    
    print(f"✅ Session created with state:")
    for key, value in initial_state.items():
        print(f"   • {key}: {value}")
    
    # Create a fresh parallel extraction agent
    parallel_agent = create_parallel_extraction_agent()
    print(f"\n🔄 Created ParallelExtractionAgent with {len(parallel_agent.sub_agents)} sub-agents:")
    for sub_agent in parallel_agent.sub_agents:
        print(f"   • {sub_agent.name}")
    
    runner = Runner(
        agent=parallel_agent,
        app_name=APP_NAME,
        session_service=session_service
    )
    
    print(f"\n⏳ Running parallel extraction... (this may take a few minutes)")
    
    query = types.Content(
        role='user', 
        parts=[types.Part(text="Extract all company information from the annual report.")]
    )
    
    events = runner.run_async(
        user_id=USER_ID, 
        session_id=f"{SESSION_ID}_parallel", 
        new_message=query
    )
    
    async for event in events:
        if event.is_final_response():
            print(f"\n✅ Parallel Extraction Complete!")
            if event.content and event.content.parts:
                print(f"📝 Final response preview:")
                result_text = event.content.parts[0].text
                print(result_text[:500] if len(result_text) > 500 else result_text)
    
    # Get updated session with all extracted data
    session = await session_service.get_session(
        app_name=APP_NAME, 
        user_id=USER_ID, 
        session_id=f"{SESSION_ID}_parallel"
    )
    
    # Extract results from session state
    results = {
        "leadership_data": session.state.get("leadership_data"),
        "metrics_data": session.state.get("metrics_data"),
        "stakeholder_data": session.state.get("stakeholder_data"),
        "products_data": session.state.get("products_data"),
        "overview_data": session.state.get("overview_data"),
    }
    
    print(f"\n{'='*60}")
    print(f"📊 EXTRACTED DATA SUMMARY")
    print(f"{'='*60}")
    for key, value in results.items():
        status = "✅ Available" if value else "❌ Not Found"
        print(f"  {key}: {status}")
        if value:
            # Show preview of the data
            preview = str(value)[:200] + "..." if len(str(value)) > 200 else str(value)
            print(f"    Preview: {preview}\n")
    
    return results

# ============================================================
# RUN IT! Execute this cell to run parallel extraction
# ============================================================
# results = await run_parallel_extraction_on_existing_store()

# Or run with custom parameters:
# results = await run_parallel_extraction_on_existing_store(
#     store_name="fileSearchStores/saksiam-store-x9i0i8eds9ex",
#     company_name="Saksiam"
# )


ImportError: cannot import name 'key_persons_agent' from 'app.agents.parallel_extraction_agent' (c:\Users\HudaGoian\Documents\Cooperate Development\Company One Pager\Company_One_Pager\app\agents\parallel_extraction_agent.py)

In [7]:
result.text

"As of December 30, 2024, the key shareholders and their ownership percentages are:\n\n*   **Mrs. Jintana Boonsalee:** 24.134%\n*   **Mr. Phoonsak Boonsalee:** 23.903%\n*   **Miss Sansanee Boonsalee:** 10.295%\n*   **Mr. Siwaphong Boonsalee:** 9.661%\n\nThe Boonsalee Family collectively holds 67.993% of the shares.\n\nOther significant shareholders include:\n*   Bualuang Ventures Company Limited: 7.156%\n*   K Mid Small Cap Equity RMF: 1.518%\n*   Mr. Nopachai Nopasuwanwong: 1.262%\n*   Thai NVDR Company Limited: 1.178%\n*   Mr. Somkiat Kraikringsri: 0.589%\n*   Mr. Aiyawatt Srivaddhanaprabha: 0.573%\n*   Minority shareholders: 12.276%\n\nBualuang Ventures Company Limited is noted as having influence over the company's policy-making or operations through the appointment of a representative to the company's board of directors."

In [11]:

# ============================================================
# CELL 3: DIRECT TOOL TEST (ask_annual_report)
# ============================================================
"""
This cell lets you test the ask_annual_report tool DIRECTLY without running agents.
Useful for debugging tool behavior with an existing vector store.
"""
import time
from typing import Dict, Any

def test_ask_annual_report_direct(store_name: str, question: str) -> Dict[str, Any]:
    """
    Direct test of the ask_annual_report logic.
    Copy of the tool function without ToolContext dependency.
    """
    if not store_name:
        print("❌ No store_name provided!")
        return {"error": "No store_name"}
    
    print(f"🔎 Querying Store '{store_name}'")
    print(f"❓ Question: {question}")
    print("-" * 60)
    
    time.sleep(2)  # Rate limiting
    
    gemini_client = client.Client(api_key=GEMINI_API_KEY)
    
    response = gemini_client.models.generate_content(
        model="gemini-2.5-flash",
        contents=question,
        config=types.GenerateContentConfig(
            tools=[types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=[store_name]
                )
            )],
        )
    )
    
    print(f"\n📝 RESPONSE TEXT:\n{'-'*40}")
    print(response.text[:2000] if len(response.text) > 2000 else response.text)
    
    # Process grounding metadata
    source_library = SourceLibrary()
    source_counter = 100
    chunk_to_source_map: Dict[int, str] = {}
    
    if response.candidates:
        candidate = response.candidates[0]
        grounding = getattr(candidate, 'grounding_metadata', None)
        
        if grounding:
            grounding_chunks = getattr(grounding, 'grounding_chunks', None)
            
            if grounding_chunks:
                print(f"\n📋 GROUNDING CHUNKS ({len(grounding_chunks)} found):")
                print("=" * 60)
                
                for i, chunk in enumerate(grounding_chunks):
                    source_id = f"src_{source_counter}"
                    source_counter += 1
                    
                    retrieved_context = getattr(chunk, 'retrieved_context', None)
                    if retrieved_context:
                        title = getattr(retrieved_context, 'title', 'Unknown')
                        text = getattr(retrieved_context, 'text', '')
                        page_number = get_page_number(text)
                        
                        source_doc = SourceDocument(
                            source_id=source_id,
                            title=title,
                            page_number=page_number,
                            raw_text=text[:500] if text else ""
                        )
                        source_library.add_source(source_doc)
                        chunk_to_source_map[i] = source_id
                        
                        print(f"\n📄 CHUNK {i} ({source_id}):")
                        print(f"   📑 Title: {title}")
                        print(f"   📖 Page: {page_number}")
                        print(f"   📝 Text: {text[:200]}...")
            
            # Check grounding supports
            grounding_supports = getattr(grounding, 'grounding_supports', None)
            if grounding_supports:
                print(f"\n📍 GROUNDING SUPPORTS ({len(grounding_supports)} found):")
                print("=" * 60)
                
                for j, support in enumerate(grounding_supports):
                    segment = getattr(support, 'segment', None)
                    chunk_indices = getattr(support, 'grounding_chunk_indices', [])
                    
                    if segment:
                        seg_text = getattr(segment, 'text', '')
                        start_idx = getattr(segment, 'start_index', 0)
                        end_idx = getattr(segment, 'end_index', 0)
                        
                        source_ids = [chunk_to_source_map.get(ci, f"chunk_{ci}") for ci in chunk_indices]
                        
                        print(f"\n📍 SUPPORT {j}:")
                        print(f"   🔢 Indices: {start_idx}-{end_idx}")
                        print(f"   🔗 Sources: {source_ids}")
                        print(f"   📝 Text: '{seg_text[:100]}...'")
    
    return {
        "response_text": response.text,
        "sources": source_library.to_dict(),
        "chunk_map": chunk_to_source_map
    }

# TEST IT! Replace with your vector store name
result = test_ask_annual_report_direct(
    store_name="fileSearchStores/saksiam-store-kwxr6sqe0z7j",
    question="Who are the key shareholders and their ownership percentages?"
)
result



🔎 Querying Store 'fileSearchStores/saksiam-store-kwxr6sqe0z7j'
❓ Question: Who are the key shareholders and their ownership percentages?
------------------------------------------------------------


2026-01-12 14:29:45.756 | DEBUG    | app.models.citation:get_page_number:32 - 📄 Extracted page number: Page 49
2026-01-12 14:29:45.757 | DEBUG    | app.models.citation:add_source:67 - 📚 Added source to library: src_100
2026-01-12 14:29:45.758 | DEBUG    | app.models.citation:get_page_number:32 - 📄 Extracted page number: Page 142
2026-01-12 14:29:45.759 | DEBUG    | app.models.citation:add_source:67 - 📚 Added source to library: src_101
2026-01-12 14:29:45.760 | DEBUG    | app.models.citation:get_page_number:32 - 📄 Extracted page number: Page 47
2026-01-12 14:29:45.761 | DEBUG    | app.models.citation:add_source:67 - 📚 Added source to library: src_102
2026-01-12 14:29:45.762 | DEBUG    | app.models.citation:get_page_number:32 - 📄 Extracted page number: Page 197
2026-01-12 14:29:45.763 | DEBUG    | app.models.citation:add_source:67 - 📚 Added source to library: src_103
2026-01-12 14:29:45.764 | DEBUG    | app.models.citation:get_page_number:32 - 📄 Extracted page number: Page 46
2026-01-12 


📝 RESPONSE TEXT:
----------------------------------------
The major shareholders and their ownership percentages as of December 30, 2024, are as follows:

1.  **Mrs. Jintana Boonsalee:** 505,857,200 shares, representing 24.134%. Mrs. Jintana Boonsalee also holds a position as Director of the Company.
2.  **Mr. Phoonsak Boonsalee:** 501,000,000 shares, representing 23.903%. Mr. Phoonsak Boonsalee also holds a position as Director of the Company.
3.  **Miss Sansanee Boonsalee:** 215,787,000 shares, representing 10.295%. Miss Sansanee Boonsalee is a Director and a major shareholder, holding 10.30% of total shares.
4.  **Mr. Siwaphong Boonsalee:** 202,500,000 shares, representing 9.661%. Mr. Siwaphong Boonsalee is a Director, Chief Executive Officer, and Managing Director, holding 9.66% of total shares.
    *   The combined shareholding of the Boonsalee Family (Mrs. Jintana, Mr. Phoonsak, Miss Sansanee, and Mr. Siwaphong) totals 1,425,144,200 shares, or 67.993%.
5.  **Bualuang Ventures Co

AttributeError: 'SourceLibrary' object has no attribute 'to_dict'

In [ ]:

# ============================================================
# CELL 4: RUN INITIAL SEARCH AGENT (Create Vector Store)
# ============================================================
async def run_initial_search():
    """Run initial_search_agent to upload PDF and create vector store."""
    session_service = InMemorySessionService()
    
    initial_state = {
        "company_name": COMPANY_NAME,
        "annual_report_filename": PDF_PATH
    }
    
    session = await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
        state=initial_state
    )
    
    print(f"📝 Session created with state: {initial_state}")
    
    runner = Runner(
        agent=initial_search_agent,
        app_name=APP_NAME,
        session_service=session_service
    )
    
    query = types.Content(role='user', parts=[types.Part(text="Find the annual report.")])
    
    events = runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=query)
    
    async for event in events:
        if event.is_final_response():
            print("\n✅ InitialSearchAgent Result:")
            if event.content and event.content.parts:
                print(event.content.parts[0].text)
    
    # Get updated session
    session = await session_service.get_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    vector_store_name = session.state.get("vector_store_name")
    
    print(f"\n✅ Vector Store Ready: {vector_store_name}")
    
    return session_service, vector_store_name

# Uncomment to run:
# session_service, vector_store_name = await run_initial_search()



In [ ]:

# ============================================================
# CELL 5: RUN EXTRACTION AGENT WITH EXISTING STORE
# ============================================================
async def run_extraction_agent(agent, agent_name: str, question: str, vector_store_name: str, session_service=None):
    """
    Run any extraction agent with an existing vector store.
    
    Args:
        agent: The agent to run (leadership_agent, stakeholder_agent, etc.)
        agent_name: Name for logging
        question: Query to send to the agent
        vector_store_name: Existing vector store name
        session_service: Optional existing session service
    """
    if session_service is None:
        session_service = InMemorySessionService()
        
        # Create session with vector store already set
        session = await session_service.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=SESSION_ID,
            state={
                "company_name": COMPANY_NAME,
                "vector_store_name": vector_store_name  # Pre-set the vector store!
            }
        )
        print(f"✅ Created session with existing vector store: {vector_store_name}")
    
    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service
    )
    
    print(f"\n{'='*60}")
    print(f"🔄 Running {agent_name}")
    print(f"{'='*60}")
    
    query = types.Content(role='user', parts=[types.Part(text=question)])
    
    events = runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=query)
    
    async for event in events:
        if event.is_final_response():
            print(f"\n✅ {agent_name} Result:")
            if event.content and event.content.parts:
                result = event.content.parts[0].text
                print(result[:3000] if len(result) > 3000 else result)
    
    # Return updated session
    session = await session_service.get_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    return session_service, session.state

# Example: Run stakeholder_agent with existing vector store
# session_service, state = await run_extraction_agent(
#     agent=stakeholder_agent(),
#     agent_name="StakeholderAgent",
#     question="Extract all shareholders and their ownership percentages.",
#     vector_store_name="fileSearchStores/saksiam-store-903a6zfw3u20"
# )



In [ ]:

# ============================================================
# CELL 6: QUICK TEST - All Agents Sequentially
# ============================================================
async def run_all_agents(vector_store_name: str):
    """Run all extraction agents in sequence with the same vector store."""
    
    agents_to_test = [
        (leadership_agent(), "LeadershipAgent", "Extract the management team information."),
        (stakeholder_agent(), "StakeholderAgent", "Extract all shareholders and their ownership percentages."),
        (metrics_agent(), "MetricsAgent", "Extract key financial metrics."),
        (business_description_agent(), "BusinessDescriptionAgent", "Describe the company's business."),
    ]
    
    results = {}
    session_service = InMemorySessionService()
    
    # Create initial session
    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
        state={
            "company_name": COMPANY_NAME,
            "vector_store_name": vector_store_name
        }
    )
    
    for agent, name, question in agents_to_test:
        try:
            _, state = await run_extraction_agent(
                agent=agent,
                agent_name=name,
                question=question,
                vector_store_name=vector_store_name,
                session_service=session_service
            )
            results[name] = "✅ Success"
        except Exception as e:
            results[name] = f"❌ Error: {e}"
            print(f"❌ {name} failed: {e}")
    
    print("\n" + "="*60)
    print("📊 SUMMARY")
    print("="*60)
    for name, status in results.items():
        print(f"  {name}: {status}")
    
    return results

# Uncomment to run all agents:
# results = await run_all_agents("fileSearchStores/saksiam-store-903a6zfw3u20")
